In [1]:
import pandas as pd
import pyodbc
import getpass
import yaml

In [2]:
with open('test.yaml','r') as f:
    config = yaml.load(f)

In [3]:
# Create connection

server = '0.0.0.0' #Weird this doesn't work with 'localhost'
database = 'test'
username = 'sa'
password = getpass.getpass()
cnxn = pyodbc.connect('DRIVER=/usr/local/lib/libmsodbcsql.17.dylib;SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()


········


For a given CSV create the 'Create Table' script. 

In [4]:
filename = config["file"]["filename"] #in config
tablename = filename[str.rfind(filename,'/')+1:].replace('.','_') # to replace the filename -> file_name_csv
df = pd.read_csv(filename)
rows = df.shape[0] # to be inserted to a metadata col
cols = df.shape[1]
fields = df.columns

print(cols)
#print(df.columns)

36


In [5]:
sql = 'DROP TABLE IF EXISTS ' + tablename
cursor.execute(sql)
cursor.commit()

In [6]:
sql = ''

for counter, field in enumerate(fields):
    last_col = len(fields) - 1
    if counter < last_col:
        sql = sql + ('['+field+']' + ' varchar(500),') + '\n'
    else:
        sql = sql + ('['+field+']' + ' varchar(500)')

sql_create = 'CREATE TABLE ' + tablename + '(' + '\n' + sql + ')'

cursor.execute(sql_create)
cursor.commit()


In [7]:
bulk_insert = 'BULK INSERT ' + tablename + ' FROM '"'/food_info.csv'"' WITH (FIELDTERMINATOR = '"','"',ROWTERMINATOR = '"'0x0a'"');'

cursor.execute(bulk_insert)
cursor.commit()


In [8]:
cnxn.close()